In [11]:
import math

import pandas as pd

import osmosys.osmosys
import activityinfo
from activityinfo import Client
from activityinfo import FormProcessing
import importlib
import model.modelAI
from activityinfo.id import generate_id
import json
from requests.exceptions import HTTPError
import os.path

importlib.reload(osmosys.osmosys)
importlib.reload(activityinfo.FormProcessing)
importlib.reload(model.modelAI)
## parameters
indicatorCodeAI = 'PR1_01'
## indicatorCodeOsmosys = 'AE0I1'
month = 'JUNIO'
month_number = '2022-06'
year = 2022

asylumPercentage = 0.3
migrationPercentage = 0.7
print('--------------------------------------PR2_02------------------------------------------------------')

--------------------------------------PR2_02------------------------------------------------------


In [12]:
# busco matchs
subformDf = osmosys.osmosys.getMatchSubforms(indicatorCodeAI)
## print(subformDf)
indicatorIdsOsmosys = subformDf.osmosys_indicador_id.unique()
subformSeries = subformDf.iloc[0]
formIdAI = subformSeries.form_id
parentFormIdAI = subformSeries.parent_form_id
indicatorIdAI = subformSeries.indicator_id
print("formIdAI: " + formIdAI)
print("parentFormIdAI: " + parentFormIdAI)
print("indicatorIdAI: " + indicatorIdAI)
print("indicatorIdsOsmosys: " + str(indicatorIdsOsmosys))

formIdAI: cxsom3tl4q3ommxdi
parentFormIdAI: cq0tf6fl4q3ommxdu
indicatorIdAI: czeze22kylzqu4i41
indicatorIdsOsmosys: [78]


In [13]:
## consulto los datos reportados
reportedDf = osmosys.osmosys.getIesPartnerCantonsByIndicatorsIdsOsmosysAndMonth(indicatorsIdsOmosys=indicatorIdsOsmosys,
                                                                                month=month, year=year)
print('reported data: ' + str(reportedDf.shape[0]))


reported data: 12


In [15]:
## obtengo los parents de AI
parentFormsIds = FormProcessing.getParentsIds(parentFormIdAI=parentFormIdAI, reportedDf=reportedDf)


In [17]:
## contruyo las estructuras de datos
changesList = []
newIds = []
for index, row in parentFormsIds.iterrows():
    # print(row)
    orgAcron = row.acronym
    cantonCode = row.canton_code
    df = osmosys.osmosys.getRefValues(year=year, month=month, orgOsmosys=orgAcron,
                                      indicatorsIdsOmosys=indicatorIdsOsmosys, cantonCode=cantonCode)
    r = df.loc[df['age_gender'] == 'NINAS'].iloc[0].value_a
    ## TODO
    ## NO HAY FORMA DE SEPARAR LOS TIPO DE SERIVICIOS, PONGO 30-70

    RM_NA = int(df.loc[df['age_gender'] == 'NINAS'].iloc[0].value_a)
    RM_NN = int(df.loc[df['age_gender'] == 'NINOS'].iloc[0].value_a)
    RM_MM = int(df.loc[df['age_gender'] == 'ADULTOS'].iloc[0].value_a)
    RM_HH = int(df.loc[df['age_gender'] == 'ADULTAS'].iloc[0].value_a)
    RM_OTR = int(df.loc[df['age_gender'] == 'OTRO'].iloc[0].value_a)
    dfDiversidad = osmosys.osmosys.getRefLgbtiDiscapacitadosValues(year=year, month=month, orgOsmosys=orgAcron,
                                                                   indicatorsIdsOmosys=indicatorIdsOsmosys,
                                                                   cantonCode=cantonCode)
    #print(dfDiversidad)
    RM_LGBT = int(dfDiversidad.loc[(dfDiversidad['diversity_type'] == 'LGBTI') & (
            dfDiversidad['country_of_origin'] == 'VENEZUELA')].iloc[0].value_a)
    RM_DS = int(dfDiversidad.loc[(dfDiversidad['diversity_type'] == 'DISCAPACITADOS') & (
            dfDiversidad['country_of_origin'] == 'VENEZUELA')].iloc[0].value_a)
    commentary = osmosys.osmosys.getCommentary(year=year, month=month, orgOsmosys=orgAcron,
                                               indicatorsIdsOmosys=indicatorIdsOsmosys).iloc[0].value_a
    PR1_01_CUAL = commentary

    PR1_01_RM_NA = int(math.ceil(RM_NA * asylumPercentage))
    PR1_01_RM_NN = int(math.ceil(RM_NN * asylumPercentage))
    PR1_01_RM_MM = int(math.ceil(RM_MM * asylumPercentage))
    PR1_01_RM_HH = int(math.ceil(RM_HH * asylumPercentage))
    PR1_01_RM_OTR = int(math.ceil(RM_OTR * asylumPercentage))
    PR1_01_RM_LGBT = int(math.ceil(RM_LGBT * asylumPercentage))
    PR1_01_RM_DS = int(math.ceil(RM_DS * asylumPercentage))
    PR1_02_RM_NA = int(math.ceil(RM_NA * migrationPercentage))
    PR1_02_RM_NN = int(math.ceil(RM_NN * migrationPercentage))
    PR1_02_RM_MM = int(math.ceil(RM_MM * migrationPercentage))
    PR1_02_RM_HH = int(math.ceil(RM_HH * migrationPercentage))
    PR1_02_RM_OTR = int(math.ceil(RM_OTR * migrationPercentage))
    PR1_02_RM_LGBT = int(math.ceil(RM_LGBT * migrationPercentage))
    PR1_02_RM_DS = int(math.ceil(RM_DS * migrationPercentage))

    orientacion = [
        "Acceso al asilo",
        "Acceso a servicios migratorios"
    ]
    subform = model.modelAI.SubFormPR1_01(mes=month_number, colltmgkykvhxgij6=indicatorIdAI,
                                          orientacion=orientacion,
                                          rmrp='Si', covid='No',
                                          poblacion_meta='Refugiados/as y migrantes',
                                          PR1_01_RM_NA=PR1_01_RM_NA,
                                          PR1_01_RM_NN=PR1_01_RM_NN,
                                          PR1_01_RM_MM=PR1_01_RM_MM, PR1_01_RM_HH=PR1_01_RM_HH,
                                          PR1_01_RM_OTR=PR1_01_RM_OTR, PR1_01_RM_LGBT=PR1_01_RM_LGBT,
                                          PR1_01_RM_DS=PR1_01_RM_DS, PR1_01_CUAL=PR1_01_CUAL,
                                          PR1_02_RM_NA=PR1_02_RM_NA,
                                          PR1_02_RM_NN=PR1_02_RM_NN,
                                          PR1_02_RM_MM=PR1_02_RM_MM,
                                          PR1_02_RM_HH=PR1_02_RM_HH,
                                          PR1_02_RM_OTR=PR1_02_RM_OTR,
                                          PR1_02_RM_LGBT=PR1_02_RM_LGBT,
                                          PR1_02_RM_DS=PR1_02_RM_DS,
                                          PR1_02_CUAL=None
                                          )
    newId = generate_id()
    newIds.append(newId)
    record = model.modelAI.Record(formId=formIdAI, recordId=newId, parentRecordId=row['@id'], fields=subform)
    changesList.append(record)
print('changes to charge: ' + str(len(changesList)))

changes to charge: 12


In [ ]:
## creo respaldo
changes = model.modelAI.Changes(changesList)
finalJson = json.dumps(changes, default=model.modelAI.default)
#open text file
try:
    os.mkdir(month)
except Exception:
    pass
text_file = open(os.path.join(month, "data_" + indicatorCodeAI + ".json"), "w")
#write string to file
n = text_file.write(finalJson)

#close file
text_file.close()
print(" creado respaldo: " + text_file.name)
newIdsDict = {"newIds": newIds}
newIdsDf = pd.DataFrame(newIdsDict)
newIdsDf.to_csv(os.path.join(month, "new_ids_" + indicatorCodeAI + ".csv"))


In [ ]:
## envio a AI
try:
    if len(changesList) > 0:
        print('se envia a AI')
        client = Client(token=osmosys.osmosys.getToken(), base_url='https://www.activityinfo.org/resources')
        client.post_resource(path='update', body=finalJson)
    else:
        print('nada que enviar a AI')
except HTTPError as e:
    code = e.response.status_code
    print('error:')
    print(code)

    print(e)